# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests
from collections import Counter

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    word_counts = Counter(text)
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    dict = {'.':'||Period||', ',':"||Comma||", '"':"||QuotationMark||", ';':"||Semicolon||", '!':"||ExclamationMark||", \
            '?':"||QuestionMark||", '(':"||LeftParentheses||", ')':"||RightParentheses||", '--':"||Dash||", '\n':"||Return||"}
    return dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

### Misc. Function

In [7]:
from tflearn.utils import get_incoming_shape

curses is not supported on this machine (please install/reinstall curses for an optimal experience)


## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.1


C:\Users\Dell\Anaconda3\envs\tensorflow-cpu\lib\site-packages\ipykernel\__main__.py:14: UserWarning: No GPU found. Please use a GPU to train your neural network.


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearingRate)`

In [9]:
def get_inputs(has_name_scope=False):
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    print("Call get_inputs: has_name_scope[%d]" % (has_name_scope))
    if has_name_scope:
        with tf.name_scope('Inputs'):
            inputs = tf.placeholder(dtype=tf.int32,shape=[None,None],name="input")
        with tf.name_scope('Targets'):
            targets = tf.placeholder(dtype=tf.int32,shape=[None,None],name="targets")
        with tf.name_scope('Learning_rate'):
            learning_rate = tf.placeholder(dtype=tf.float32,name="learning_rate")
    else:
        inputs = tf.placeholder(dtype=tf.int32,shape=[None,None],name="input")
        targets = tf.placeholder(dtype=tf.int32,shape=[None,None],name="targets")
        learning_rate = tf.placeholder(dtype=tf.float32,name="learning_rate")
    return (inputs, targets, learning_rate)

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Call get_inputs: has_name_scope[0]
Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [10]:
def get_init_cell(batch_size, rnn_size, lstm_layers=1, has_name_scope=False):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
   # print(batch_size)
    print("Call get_init_cell: rnn_size[%d] lstm_layers[%d] has_name_scope[%d]" % \
          (rnn_size, lstm_layers, has_name_scope))
    if has_name_scope:
        with tf.name_scope('RNN_Layers'):
            lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
            cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
        with tf.name_scope('RNN_init_state'):
            initial_state = cell.zero_state(batch_size, tf.float32)
            initial_state = tf.identity(initial_state,name="initial_state")
    else:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        cell = tf.contrib.rnn.MultiRNNCell([lstm] * lstm_layers)
        initial_state = cell.zero_state(batch_size, tf.float32)
        initial_state = tf.identity(initial_state,name="initial_state")
    
    return cell, initial_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Call get_init_cell: rnn_size[256] lstm_layers[1] has_name_scope[0]
Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [11]:
def get_embed(input_data, vocab_size, embed_dim=100, has_name_scope=False):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    print("Call get_embed: vocab_size[%d] embed_dim[%d] has_name_scope[%d]" % \
          (vocab_size, embed_dim, has_name_scope))
    embed_init = tf.truncated_normal(shape=[vocab_size,embed_dim],stddev=.1)
    #  F_W = tf.Variable(xavier_init(shape_W, conv_input_size, conv_output_size))
    if has_name_scope:
        with tf.name_scope('Embedding'):
            embedding = tf.Variable(embed_init)
            embed = tf.nn.embedding_lookup(embedding,input_data, name="embedding")
    else:
        embedding = tf.Variable(embed_init)
        embed = tf.nn.embedding_lookup(embedding,input_data, name="embedding")
    return embed


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Call get_embed: vocab_size[27] embed_dim[256] has_name_scope[0]
Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [48]:
def build_rnn(cell, inputs, has_name_scope=False):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    print("Call build_rnn: cell.state_size[%s] cell.output_size[%s] has_name_scope[%d]" % \
          (cell.state_size, cell.output_size, has_name_scope))
    if has_name_scope:
        with tf.name_scope('RNN_forward'):
            outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
            final_state = tf.identity(final_state,name="final_state")
    else:
        outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
        final_state = tf.identity(final_state,name="final_state")
    return outputs, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Call build_rnn: cell.state_size[(LSTMStateTuple(c=256, h=256), LSTMStateTuple(c=256, h=256))] cell.output_size[256] has_name_scope[0]
final_state:0
Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [13]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim=100, has_name_scope=False):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim, has_name_scope=has_name_scope)
    rnn_outputs, final_state = build_rnn(cell, embed, has_name_scope=has_name_scope)
    if has_name_scope:
        with tf.name_scope('Logits') as fc_scope:
            predictions = tf.contrib.layers.fully_connected(inputs=rnn_outputs, num_outputs=vocab_size, activation_fn=None,\
                                                           weights_initializer=tf.contrib.layers.xavier_initializer(uniform=False))
    else:
        predictions = tf.contrib.layers.fully_connected(inputs=rnn_outputs, num_outputs=vocab_size, activation_fn=None,\
                                                       weights_initializer=tf.contrib.layers.xavier_initializer(uniform=False))

    print("build_nn: rnn_size=%d vocab_size=%d embed_dim=%d" % (rnn_size,vocab_size,embed_dim))
    print("build_nn: cell.state_size",cell.state_size)
    print("build_nn: cell.output_size",cell.output_size)
    print("build_nn: input_data",get_incoming_shape(input_data))
    print("build_nn: embed",get_incoming_shape(embed))
    print("build_nn: rnn_outputs",get_incoming_shape(rnn_outputs))
    print("build_nn: final_state",get_incoming_shape(final_state))
    print("build_nn: predictions",get_incoming_shape(predictions))
    
    if has_name_scope:
        return predictions, final_state, fc_scope
    else:
        return predictions, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Call get_embed: vocab_size[27] embed_dim[100] has_name_scope[0]
Call build_rnn: cell.state_size[(LSTMStateTuple(c=256, h=256), LSTMStateTuple(c=256, h=256))] cell.output_size[256] has_name_scope[0]
build_nn: rnn_size=256 vocab_size=27 embed_dim=100
build_nn: cell.state_size (LSTMStateTuple(c=256, h=256), LSTMStateTuple(c=256, h=256))
build_nn: cell.output_size 256
build_nn: input_data [128, 5]
build_nn: embed [128, 5, 100]
build_nn: rnn_outputs [128, 5, 256]
build_nn: final_state [2, 2, None, 256]
build_nn: predictions [128, 5, 27]
Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [16]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    text_len = len(int_text)
    slice_size = batch_size * seq_length
    num_of_batch = text_len // slice_size
    
    # Drop the last few characters to make only full batches
    x = np.asarray(int_text[: num_of_batch*slice_size])
    y = np.asarray(int_text[1: num_of_batch*slice_size + 1])
    x = np.stack(np.hsplit(np.stack(np.split(x,num_of_batch)),batch_size),axis=1)
    y = np.stack(np.hsplit(np.stack(np.split(y,num_of_batch)),batch_size),axis=1)
    r = np.stack((x,y),axis=1)
    
    print("Call get_batches: text=%d num_of_batch=%d batch_size=%d seq_length=%d slice_size=%d -> %s" % 
          (text_len,num_of_batch,batch_size,seq_length,slice_size,r.shape))
    
    return r


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Call get_batches: text=5000 num_of_batch=7 batch_size=128 seq_length=5 slice_size=640 -> (7, 2, 128, 5)
Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [17]:
# Number of Epochs
num_epochs = 256
# Batch Size
batch_size = 128
# RNN Size
rnn_size = 256
# Sequence Length
seq_length = 100
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 8
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [69]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

Call get_inputs: has_name_scope[0]
Call get_init_cell: rnn_size[256] lstm_layers[1] has_name_scope[0]
Call get_embed: vocab_size[6779] embed_dim[100] has_name_scope[0]
Call build_rnn: cell.state_size[(LSTMStateTuple(c=256, h=256),)] cell.output_size[256] has_name_scope[0]
final_state:0
build_nn: rnn_size=256 vocab_size=6779 embed_dim=100
build_nn: cell.state_size (LSTMStateTuple(c=256, h=256),)
build_nn: cell.output_size 256
build_nn: input_data [None, None]
build_nn: embed [None, None, 100]
build_nn: rnn_outputs [None, None, 256]
build_nn: final_state [1, 2, None, 256]
build_nn: predictions [None, None, 6779]


- get_inputs(has_name_scope=False)
- get_init_cell(batch_size, rnn_size, lstm_layers=1, has_name_scope=False)
- get_embed(input_data, vocab_size, embed_dim, has_name_scope=False)
- build_rnn(cell, inputs, has_name_scope=False)
- build_nn(cell, rnn_size, input_data, vocab_size, embed_dim=300, has_name_scope=False)
- get_batches(int_text, batch_size, seq_length)

In [18]:
from tensorflow.contrib import seq2seq

def build_seq_network(vocab_size, lstm_layers=1, embed_dim=100, has_name_scope=False):
    tf.reset_default_graph()
    train_graph = tf.Graph()
    with train_graph.as_default():
        input_text, targets, lr = get_inputs(has_name_scope=has_name_scope)
        input_data_shape = tf.shape(input_text)
        cell, initial_state = get_init_cell(input_data_shape[0], rnn_size, lstm_layers, has_name_scope=has_name_scope)
        logits, final_state, fc_scope = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim, has_name_scope=has_name_scope)

        # Probabilities for generating words
        if has_name_scope:
            with tf.name_scope(fc_scope):
                probs = tf.nn.softmax(logits, name='probs')
        else:
            probs = tf.nn.softmax(logits, name='probs')

        # Loss function
        if has_name_scope:
            with tf.name_scope('Cost'):
                cost = seq2seq.sequence_loss(logits,targets,
                    tf.ones([input_data_shape[0], input_data_shape[1]]),name="cost")
        else:
            cost = seq2seq.sequence_loss(logits,targets,
                tf.ones([input_data_shape[0], input_data_shape[1]]),name="cost")

        if has_name_scope:
            with tf.name_scope('Optimizer'):
                # Optimizer
                optimizer = tf.train.AdamOptimizer(lr)
                # Gradient Clipping
                gradients = optimizer.compute_gradients(cost)
                capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
                train_op = optimizer.apply_gradients(capped_gradients)
        else:
            # Optimizer
            optimizer = tf.train.AdamOptimizer(lr)
            # Gradient Clipping
            gradients = optimizer.compute_gradients(cost)
            capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
            train_op = optimizer.apply_gradients(capped_gradients)

        return (train_graph, input_text, targets, lr, initial_state, final_state, cost, train_op)

In [21]:
vocab_size = len(int_to_vocab)
train_graph, input_text, targets, lr, initial_state, final_state, cost, train_op = \
build_seq_network(vocab_size,lstm_layers=1,has_name_scope=True)
with tf.Session(graph=train_graph) as sess:
    print(sess.graph)
    file_writer = tf.summary.FileWriter('./logs/1', sess.graph)

Call get_inputs: has_name_scope[1]
Call get_init_cell: rnn_size[256] lstm_layers[1] has_name_scope[1]
Call get_embed: vocab_size[6779] embed_dim[100] has_name_scope[1]
Call build_rnn: cell.state_size[(LSTMStateTuple(c=256, h=256),)] cell.output_size[256] has_name_scope[1]
build_nn: rnn_size=256 vocab_size=6779 embed_dim=100
build_nn: cell.state_size (LSTMStateTuple(c=256, h=256),)
build_nn: cell.output_size 256
build_nn: input_data [None, None]
build_nn: embed [None, None, 100]
build_nn: rnn_outputs [None, None, 256]
build_nn: final_state [1, 2, None, 256]
build_nn: predictions [None, None, 6779]


## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [22]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Call get_batches: text=69100 num_of_batch=5 batch_size=128 seq_length=100 slice_size=12800 -> (5, 2, 128, 100)
Epoch   0 Batch    0/5   train_loss = 8.821
Epoch   1 Batch    3/5   train_loss = 6.079
Epoch   3 Batch    1/5   train_loss = 5.906
Epoch   4 Batch    4/5   train_loss = 5.941
Epoch   6 Batch    2/5   train_loss = 5.556
Epoch   8 Batch    0/5   train_loss = 5.347
Epoch   9 Batch    3/5   train_loss = 5.205
Epoch  11 Batch    1/5   train_loss = 4.848
Epoch  12 Batch    4/5   train_loss = 4.777
Epoch  14 Batch    2/5   train_loss = 4.425
Epoch  16 Batch    0/5   train_loss = 4.284
Epoch  17 Batch    3/5   train_loss = 4.231
Epoch  19 Batch    1/5   train_loss = 3.974
Epoch  20 Batch    4/5   train_loss = 3.986
Epoch  22 Batch    2/5   train_loss = 3.743
Epoch  24 Batch    0/5   train_loss = 3.627
Epoch  25 Batch    3/5   train_loss = 3.638
Epoch  27 Batch    1/5   train_loss = 3.389
Epoch  28 Batch    4/5   train_loss = 3.377
Epoch  30 Batch    2/5   train_loss = 3.208
Epoch  32

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [23]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [82]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()
print(load_dir)
print(save_dir)

./save
./save


## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [55]:
def get_tensors(loaded_graph, has_name_scope=False):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    if has_name_scope:
        inputs = loaded_graph.get_tensor_by_name("Inputs/input:0")
        initial_state = loaded_graph.get_tensor_by_name("RNN_init_state/initial_state:0") 
        final_state = loaded_graph.get_tensor_by_name("RNN_forward/final_state:0") 
        probs = loaded_graph.get_tensor_by_name("Logits/probs:0")
    else:
        inputs = loaded_graph.get_tensor_by_name("input:0")
        initial_state = loaded_graph.get_tensor_by_name("initial_state:0") 
        final_state = loaded_graph.get_tensor_by_name("final_state:0") 
        probs = loaded_graph.get_tensor_by_name("probs:0")
    
    return  (inputs, initial_state, final_state, probs)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)


Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [56]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    word_int = np.random.choice(len(int_to_vocab.keys()), p=probabilities)
    return int_to_vocab[word_int]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [78]:
# vocab_size = len(int_to_vocab)
# loaded_graph, input_text, targets, lr, initial_state, final_state, cost, train_op = \
# build_seq_network(vocab_size,lstm_layers=1,has_name_scope=False)
loaded_graph = tf.Graph()
get_tensors(loaded_graph,True)

KeyError: "The name 'Inputs/input:0' refers to a Tensor which does not exist. The operation, 'Inputs/input', does not exist in the graph."

In [79]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# loaded_graph = tf.Graph()
loaded_graph = train_graph
hns = False
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph,has_name_scope=hns)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

FailedPreconditionError: Attempting to use uninitialized value Variable
	 [[Node: Variable/read = Identity[T=DT_FLOAT, _class=["loc:@Variable"], _device="/job:localhost/replica:0/task:0/cpu:0"](Variable)]]

Caused by op 'Variable/read', defined at:
  File "C:\Users\Dell\Anaconda3\envs\tensorflow-cpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Dell\Anaconda3\envs\tensorflow-cpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Dell\Anaconda3\envs\tensorflow-cpu\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\Dell\Anaconda3\envs\tensorflow-cpu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Dell\Anaconda3\envs\tensorflow-cpu\lib\site-packages\ipykernel\kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "C:\Users\Dell\Anaconda3\envs\tensorflow-cpu\lib\site-pac

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.